# Cybercomp Example - Neuroscience

Here, multiplicity (e.g., Grid Search) should be explicitly defined

## Set Up Code Completions

In [ ]:
from pathlib import Path
from cybercomp import API

# generate statically typed objects to define experiments
api = API(server_url="http://127.0.0.1:8765", base_path=Path("./completions"))
api.sync()

Connected to cybercomp server on http://127.0.0.1:8765


[Sources] : 100%|██████████| 3/3 [00:00<00:00, 1227.48it/s, tunnel_effect]

Completions generated at: /Users/yasith/projects/cybercomp/demos/completions


## Import Cybercomp APIs

In [2]:
from cybercomp.functional import experiment, hyperparameter, parameter
from cybercomp.functional import fetch
from cybercomp.runtimes import LocalRuntime

from completions import engines, models
from completions.types import *

## Exploring Available Models

In [3]:
import completions.models.sleep_stage_transition as sst

sst.build.describe()
sst.network_create.describe()
sst.network_update.describe()
sst.simulate.describe()
display()

* [Model] build
  - params?: 191 ['P[~rem_scale](0.9)', 'P[~FS1_alpha](3.8)', 'P[~IA_TC_G_A](1)', 'P[~RS_v_DEND](-60)', 'P[~RS_sigma_dc](1.0)', 'P[~awake_GABAd2](0.22)', 'P[~Ih_TC_cac](0.0015)', 'P[~awake_AMPAd2_fix](0.2)', 'P[~RS_sigma_e](1.0)', 'P[~gk_cx_slow_rem](1.0)', 'P[~RS_spike](0)', 'P[~TCcore_memory_GB_K2](0.0012)', 'P[~I_HH](1)', 'P[~INsynCore_DEND_G_Nap](0.0)', 'P[~INsynMatrix_DEND_G_Nap](0.0)', 'P[~FS1_sigma_ii](0.5)', 'P[~Ih_TC_pc](0.01)', 'P[~gk_cx_slow_s2](1.0)', 'P[~TCcore_E_l](-70)', 'P[~Ih_TC_ginc](1.5)', 'P[~GABA_B_K1](0.52)', 'P[~TCcore_memory_GB_K3](0.1)', 'P[~GB_C](0)', 'P[~awake_AMPAd2](0.2)', 'P[~FS1_ii](0)', 'P[~awake_GABAd2_fix](0.22)', 'P[~INap_CX_Sig](5)', 'P[~TCcore_D](2)', 'P[~CXsyn6_E_l](-68)', 'P[~GABA_A_R1](0)', 'P[~gk_cx_slow_s3](1.0)', 'P[~RS_beta_e](0.133)', 'P[~gkl_TC_awake_fix](0.79)', 'P[~GABA_B_Cdur](0.3)', 'P[~INap_CX_Q10](2.7)', 'P[~TCmatrix_VtrK](-28)', 'P[~GABA_A_Beta](0.166)', 'P[~INsynCore_G_Km](0.02)', 'P[~INaK_VtrK](-50)', 'P[~TAU_Map](0

## Defining a Multi-Step Experiment

Here, you can define individual experiments (i.e., steps) and chain them into larger experiments, if need be.
Internally, Cybercomp will connect the outputs from previous experiments into subsequent experiments that depend on them.
Users only need to provide the non-inferrable parameters / hyperparameters.

In [ ]:
exp0 = experiment(sst.build(), engines.mustache(), name="make_source")
exp0.setup(
    {
        sst.build.template("sleep_stage_transition"),
        sst.build.overrides("test.yml"),
    }
)
exp0.describe()

with LocalRuntime(True) as r:
    exp0.run_sync(r)
    result = exp0.fetch(r, {sst.build.code})
print("* [Results]", list(map(str, result)))

* [Setup] Experiment=make_source
  * [Setup] Step=make_source
    * [Model] build
      - params?: 191 ['P[~rem_scale](0.9)', 'P[~FS1_alpha](3.8)', 'P[~IA_TC_G_A](1)', 'P[~RS_v_DEND](-60)', 'P[~RS_sigma_dc](1.0)', 'P[~awake_GABAd2](0.22)', 'P[~Ih_TC_cac](0.0015)', 'P[~awake_AMPAd2_fix](0.2)', 'P[~RS_sigma_e](1.0)', 'P[~gk_cx_slow_rem](1.0)', 'P[~RS_spike](0)', 'P[~TCcore_memory_GB_K2](0.0012)', 'P[~I_HH](1)', 'P[~INsynCore_DEND_G_Nap](0.0)', 'P[~INsynMatrix_DEND_G_Nap](0.0)', 'P[~FS1_sigma_ii](0.5)', 'P[~Ih_TC_pc](0.01)', 'P[~gk_cx_slow_s2](1.0)', 'P[~TCcore_E_l](-70)', 'P[~Ih_TC_ginc](1.5)', 'P[~GABA_B_K1](0.52)', 'P[~TCcore_memory_GB_K3](0.1)', 'P[~GB_C](0)', 'P[~awake_AMPAd2](0.2)', 'P[~FS1_ii](0)', 'P[~awake_GABAd2_fix](0.22)', 'P[~INap_CX_Sig](5)', 'P[~TCcore_D](2)', 'P[~CXsyn6_E_l](-68)', 'P[~GABA_A_R1](0)', 'P[~gk_cx_slow_s3](1.0)', 'P[~RS_beta_e](0.133)', 'P[~gkl_TC_awake_fix](0.79)', 'P[~GABA_B_Cdur](0.3)', 'P[~INap_CX_Q10](2.7)', 'P[~TCmatrix_VtrK](-28)', 'P[~GABA_A_Beta](0.1

In [ ]:
# @giri: create_model example
# M = create_model(‘g++ -O2 generate_network.cpp -o generete _network’, params_file = ‘networks.cfg’
# M.describe() - prints out the entire spec (params, default values)

In [ ]:
exp0 = experiment(sst.build(), engines.mustache(), name="make_source")
exp1 = experiment(sst.network_create(), engines.cpp(), name="make_network")
exp2 = experiment(sst.network_update(), engines.python(), name="update_network")
exp3 = experiment(sst.simulate(), engines.cpp(), name="simulate_network")

multi_step_exp = exp0 >> exp1 >> exp2 >> exp3
multi_step_exp.setup(
    {
        sst.build.template("sleep_stage_transition"),
        sst.network_update.network_config("test.yml"),
    }
)
multi_step_exp.describe()

with LocalRuntime(True) as r:
    multi_step_exp.run_sync(r)
    result = multi_step_exp.fetch(r, {sst.simulate.time_cx, sst.simulate.time_tca})
print("* [Results]", list(map(str, result)))

In [ ]:
with LocalRuntime(True) as r:

    exp0 = experiment(sst.build(), engines.mustache(), name="make_source")
    exp0.setup({sst.build.template("sleep_stage_transition")})
    exp0.run_sync(r)
    res0 = multi_step_exp.fetch(r)

    exp1 = experiment(sst.network_create(), engines.cpp(), name="make_network")
    exp1.setup({*res0.values()})
    exp1.run_sync(r)
    res1 = multi_step_exp.fetch(r)

    exp2 = experiment(sst.network_update(), engines.python(), name="update_network")
    exp2.setup({*res0.values(), *res1.values()})
    exp2.run_sync(r)
    res2 = multi_step_exp.fetch(r, {sst.simulate.time_cx, sst.simulate.time_tca})

    exp3 = experiment(sst.simulate(), engines.cpp(), name="simulate_network")
    exp3.setup({*res0.values(), *res1.values(), *res2.values()})
    exp3.run_sync(r)
    res3 = multi_step_exp.fetch(r, {sst.simulate.time_cx, sst.simulate.time_tca})

print("* [Results]", list(map(str, result)))

In [6]:
import argparse

def create_dynamic_class():

    return type(
        "Test",
        (argparse.Namespace,),
        {
            "foo": "default_foo",
            "bar": "default_bar",
            "__annotations__": {"foo": str, "bar": str},  # Type hinting for Pylance
        },
    )


# Create the dynamic class
Test = argparse.Namespace
x = Test(foo=3, bar=5)
print(x.foo)
print(x.bar)

3
5


In [7]:
for i in range(0, 100):
    # large_experiment = (exp0 >> exp1 >> exp2)
    large_exp_A.params.build.awake_GABA_TC_fix = i
    sst.build.awake_GABA_TC_fix = i
    large_experiment.run()

wait_for_completion()

# large_experiment = (preprocess >> exp1 >> exp2)
large_experiment.describe()

NameError: name 'large_exp_A' is not defined

## Defining a New Experiment from Scratch

Here, you can define an experiment outside of the completions database, by specifying the following,
(a) the command which to run
(b) the engines needed for the command to run
(c) the parameters needed for the command to run

## Running the Experiment

First, let's define the runtime for this experiment. For this example, we will use the local runtime (i.e., the machine this notebook will be executed on).

In [ ]:
# run experiment
with LocalRuntime(reuse_past_runs=True) as r:
    ref = run(
        large_experiment,
        [
            parameter(connection_info, "samples/sleep_stage_transition/connection_info"),
            parameter(current_params, "samples/sleep_stage_transition/params.txt"),
        ],
        runtime=r,
    )
    # fetch an output from experiment execution
    [obs] = fetch(*ref, runtime=r, observables=[INsynCore_DEND_G_Nap, awake_GABA_TC])

    obs[connection_info].save("data/connection_info")

### Running a Secondary Experiment

Here, we will use the outputs from the previous experiment run to define and run a second experiment.

In [ ]:
with LocalRuntime(reuse_past_runs=True) as r:
    # prepare a new experiment with the fetched output as a parameter
    ref = run(
        experiment(sst.network_update(), engines.python(), name="update_network"),
        [
            parameter(connection_info, "data/connection_info"),
            parameter(network_config, "data/network.cfg"),
        ],
        runtime=r,
    )
    # fetch an output from experiment execution
    [obs] = fetch(*ref, runtime=r, observables=[connection_info])
    obs[connection_info].save("data/connection_info_updated")

In [ ]:
# run a grid search on s2, using output from s1
s1 = experiment(models.neuro_create_network(), engines.neuro_sleep_stage(), name="make_network")
s1.describe()

s1.run()

s2 = experiment(models.sample_make_apples(), engines.neuro_sleep_stage(), name="simulate_network")

s1 = ['x', 'y'] -> ['neuro_connection_info']
s2 = ['neuro_connection_info', 'a'] -> ['d']

s1b = experiment(models.modify_network(), engines.neuro_sleep_stage(), name="modify_network")
s = (s1 >> s1b)

# 'a' = "val_a"
# 'b' = "val_b"
# s2 = ['a', 'b', 'neuro_connection_info'] -> ['d']

# as a requirement, s2 should take in a 'neuro_connection_info' as an input

s1 >> s2 >> s3

['z', 'd']

exp = (s1 >> s2)

# args
args = (
    parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
    parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"),
    hyperparameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
)

In [ ]:
%run_remote

output = ex1 > ex2 > ex3

In [ ]:
%run_local

output > ex4

In [ ]:
range_P = [
    P.Range(models.stim_current.neuro_connection_info, 0.1, 1.0, 10),
    P.Range(models.stim_current.neuro_current_params, 0.1, 1.0, 10),
]
range_H = [
    H.Range(engines.bazhenov_labs_engine.neuro_current_params, 0.1, 1.0, 10),
]

In [ ]:
sweep1 = Experiment.Sweep("bazhenov_network_grid_search", (seq1,), range_P, range_H, []) # change syntax
sweep1.run(runtime)
exChain_obs = sweep1.fetch(runtime)

In [ ]:
# run sweep
for p in range_P:
    for h in range_H:
        

In [ ]:
ex12 = Collection(
    name="simulate_bhazenov_network",
    models=[models.make_network, models.stim_current],
    engine=engines.bazhenov_labs_engine,
    parameters=[
        models.make_network.neuro_network_config("samples/sleep_stage_transition/params.txt"),
        models.stim_current.neuro_current_params("samples/sleep_stage_transition/connection_info")
    ],
    parameters={
        "neuro_network_config": P("samples/sleep_stage_transition/network.cfg"),
        "neuro_current_params": P("samples/sleep_stage_transition/params.txt"),
    },
    hyperparameters={
        "neuro_current_params": H("samples/sleep_stage_transition/params.txt"),
    },
    observables={
        "neuro_time_cx6": O("observables/time_cx6")
    },
)

runtime = Runtime()
ex2.execute(runtime)
ex2_obs = ex2.gather_observables(runtime)

## POC - double pendulum simulation
## Real - replicate simone's experiments


# the intersecting observables are piped into the params of the next experiment
# only the intersecting observables are shown in autocompletion

experiment_2 = create_new_experiment_from(experiment_1)
experiment_2.name = "<>"
experiment_2.parameters["neuro/network_config"] = "new-value"
experiment.check_existing()

experiment.observables

experiemnt.validate()
experiment.run(
hpc_recipe={}
)

# !!! [collection] - a set of experiments with common observables

# observables may be a huge list, so need not provide everytime when its

implictly discoverable

# to get experiments run with different observables

collection = create_collection(
model="name_of_model",
parameters={
"neuro/network_config": [],
},
)

# the collection experiments are pulled from the db

collection = create_collection(
model=["model1", "model2", ...],
parameters={
"neuro/network_config": [],
},
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

# all experiments sharing the same observables

collection = create_collection(
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

collection = [experiment_1, experiment_2]

# experiment collection

# example of experiment chaining (top-to-bottom mro)

# example 1

experiment = Experiment(
experiment_1,
experiment_2,
)

# example 2

experiment = Experiment(
[experiment_2, experiment_3, .....], #
experiment_1, #
)

#

[
exp2 -> exp1,
exp3 -> exp1,
]

# example 3

experiment = Experiment(
[experiment_2, ...collection.experiments],
experiment_1,
)

In [ ]:
# analysis part =========================

# takes a collection as input,

# and runs some function over the observables on that

# collection

# a primitive form of experiment using a collection of experiments as input

analysis = Analysis(
collection=[],
function={

    }

)

analysis = experiment